## Создаем соединение с базой данных с помощью библиотеки sqlite3

In [1]:
import sqlite3
import pandas as pd

sql_connection=sqlite3.connect('../../data/checking-logs.sqlite')
sql_connection

## Получаем схему таблицы test.

In [2]:
pd.io.sql.read_sql('PRAGMA table_info(test);', sql_connection)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Получаем первые 10 строк таблицы test.

In [3]:
limit=pd.io.sql.read_sql('SELECT * FROM test LIMIT 10;', sql_connection)
limit

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45,2020-04-18 10:56:55
1,user_30,laba04,2020-04-18 13:36:53,2020-04-17 22:46:26
2,user_30,laba04s,2020-04-18 14:51:37,2020-04-17 22:46:26
3,user_14,laba04,2020-04-18 15:14:00,2020-04-18 10:53:52
4,user_14,laba04s,2020-04-18 22:30:30,2020-04-18 10:53:52
5,user_19,laba04,2020-04-20 19:05:01,2020-04-21 20:30:38
6,user_25,laba04,2020-04-20 19:16:50,2020-05-09 23:54:54
7,user_21,laba04,2020-04-21 17:48:00,2020-04-22 22:40:36
8,user_30,project1,2020-04-22 12:36:24,2020-04-17 22:46:26
9,user_21,laba04s,2020-04-22 20:09:21,2020-04-22 22:40:36


## Находим среди всех пользователей минимальное значение дельты (периода времени от даты первого коммита пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос. Название итогового датафрейма — df_min

* Для этого сджойним данные таблицы с данными таблицы deadlines.
* Период времени отображается в часах.
* Не учитываем лабораторное задание project1 с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
* Сохраняем значение в датафрейме **df_min** с соответствующим uid

In [4]:
df_min = """
SELECT uid, MIN((strftime('%s', first_commit_ts) - deadlines) / 3600) AS 'min(diff)'
FROM
    (SELECT * FROM test WHERE labname NOT LIKE ("project1"))
LEFT JOIN
    deadlines ON labname = deadlines.labs;"""

df_min = pd.io.sql.read_sql(df_min, sql_connection)
df_min

,uid,min(diff)
0,user_30,-202


## Выполняем те же самые операции для максимального значения дельты, используя только один запрос. Название итогового датафрейма — df_max

In [5]:
df_max = """
SELECT uid, MAX((strftime('%s', first_commit_ts) - deadlines) / 3600) AS 'max(diff)'
FROM
    (SELECT * FROM test WHERE labname NOT LIKE ("project1"))
LEFT JOIN
    deadlines ON labname = deadlines.labs;"""

df_max = pd.io.sql.read_sql(df_max, sql_connection)
df_max

,uid,max(diff)
0,user_25,-2


## Выполняем те же самые операции для среднего значения дельты, используя только один запрос. На этот раз итоговый датафрейм не включает столбец uid; имя датафрейма — df_avg.

In [6]:
df_avg ="""
SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS 'AVG(diff)'
FROM
    (SELECT * FROM test WHERE labname NOT LIKE ("project1"))
LEFT JOIN
    deadlines ON labname = deadlines.labs;"""
    
df_avg = pd.io.sql.read_sql(df_avg, sql_connection)
df_avg

,AVG(diff)
0,-89.125


## Проверяем предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого рассчитываем коэффициент корреляции между количеством просмотров страниц и разницей между датами.

1. Используя только один запрос, создаем таблицу со столбцами: uid, avg_diff, pageviews.


 * uid — это uid, существующие в таблице test.

 * avg_diff — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.

 * pageviews — количество посещений ленты новостей одним пользователем.


2. Не учитывем лабораторное задание project1.
3. Сохраняем результаты в датафрейме views_diff.
4. Используем метод corr() библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.

In [7]:
views_diff = """
SELECT
    test.uid, avg_diff, pageviews.pageviews
FROM (
    SELECT uid, AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
    FROM test
    LEFT JOIN deadlines ON test.labname = deadlines.labs
    WHERE labname NOT LIKE 'project1'
    GROUP BY uid
) test
LEFT JOIN (
    SELECT uid, COUNT(*) AS pageviews
    FROM pageviews
    WHERE uid LIKE 'user_%'
    GROUP BY uid
) pageviews ON test.uid = pageviews.uid;
"""

views_diff = pd.read_sql(views_diff, sql_connection)
views_diff

views_diff[["avg_diff", "pageviews"]].corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## Закрываем соединение с базой данных.

In [8]:
sql_connection.close()